# Fitness Assistant

In [303]:
import os  
from dotenv import load_dotenv 

from langchain_groq import ChatGroq
import re
import httpx

load_dotenv()

True

In [304]:
LANGCHAIN_API_KEY = os.getenv("LANGCHAIN_API_KEY")
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")
GROQ_API_KEY = os.getenv("GROQ_API_KEY")
LANGCHAIN_PROJECT = os.getenv("LANGCHAIN_PROJECT")

In [305]:
os.environ["LANGCHAIN_API_KEY"] = LANGCHAIN_API_KEY
os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
os.environ["GROQ_API_KEY"] = GROQ_API_KEY
os.environ["LANGCHAIN_PROJECT"] = LANGCHAIN_PROJECT
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"

In [306]:
llm = ChatGroq(model="meta-llama/llama-4-scout-17b-16e-instruct")

In [307]:
llm.invoke('hi').content

'Hi! How are you today? Is there something I can help you with or would you like to chat?'

In [308]:
message = [
    {
        "role" : "system",
        "content" : "You are a helpful assistant."
    },
    {
        "role" : "user",
        "content" : "Hi, how are you ?"
    }
]

In [309]:
result = llm.invoke(message)

In [310]:
print(result.content)

I'm doing well, thanks for asking! I'm a large language model, so I don't have feelings like humans do, but I'm always happy to chat with you and help with any questions or topics you'd like to discuss. How about you? How's your day going so far?


In [311]:
class Chatbot():
    def __init__(self,system_content=""):
        self.system_content = system_content
        self.message = []
        if self.system_content:
            self.message.append({"role" : "system", "content" : system_content})
    def __call__(self,message):
        self.message.append({"role" : "user", "content" : message})
        result = self.execute()
        self.message.append({"role" : "assistant", "content" : result})
        return result
    def execute(self):
        llm = ChatGroq(model="gemma2-9b-it")
        result = llm.invoke(self.message)
        return result.content
    

In [312]:
chat = Chatbot(system_content="You are a helpful assistant")

In [313]:
chat("Hello, I am DP.")

"Hello DP, it's nice to meet you!\n\nWhat can I do to help you today? 😄  \n\n"

In [314]:
chat.message

[{'role': 'system', 'content': 'You are a helpful assistant'},
 {'role': 'user', 'content': 'Hello, I am DP.'},
 {'role': 'assistant',
  'content': "Hello DP, it's nice to meet you!\n\nWhat can I do to help you today? 😄  \n\n"}]

In [315]:
prompt = """You run in a loop of Thought, Action, PAUSE, Observation.
At the end of the loop, you output an Answer related to fitness, health, or exercise.
Use Thought to describe your thoughts about the question you have been asked.
Use Action to run one of the actions available to you - then return PAUSE.
Observation will be the result of running those actions.

Your available actions are:

calculate:
e.g. calculate: 1800 * 0.2
Runs a calculation (e.g., calorie intake, BMI, workout progress) and returns the result.

fitness_wiki:
e.g. fitness_wiki: Benefits of Cardio
Returns a summary of fitness-related information from a fitness knowledge base (Wikipedia or similar).

exercise_plan:
e.g. exercise_plan: Beginner full-body workout
Returns a recommended workout plan based on the type of exercise or user level.

nutrition_info:
e.g. nutrition_info: Protein sources for vegetarians
Looks up nutritional information or suggestions for diets.

Example session:
Question: What are the benefits of cardio exercises?
Thought: I should look up information about the benefits of cardio exercises.
Action: fitness_wiki: Benefits of Cardio
PAUSE

You will be called again with this:
Observation: Cardio exercises improve cardiovascular health, increase lung capacity, and aid in weight loss

Then output:
Answer: Cardio exercises improve cardiovascular health, increase lung capacity, and help with weight loss
""".strip()

In [316]:
prompt

'You run in a loop of Thought, Action, PAUSE, Observation.\nAt the end of the loop, you output an Answer related to fitness, health, or exercise.\nUse Thought to describe your thoughts about the question you have been asked.\nUse Action to run one of the actions available to you - then return PAUSE.\nObservation will be the result of running those actions.\n\nYour available actions are:\n\ncalculate:\ne.g. calculate: 1800 * 0.2\nRuns a calculation (e.g., calorie intake, BMI, workout progress) and returns the result.\n\nfitness_wiki:\ne.g. fitness_wiki: Benefits of Cardio\nReturns a summary of fitness-related information from a fitness knowledge base (Wikipedia or similar).\n\nexercise_plan:\ne.g. exercise_plan: Beginner full-body workout\nReturns a recommended workout plan based on the type of exercise or user level.\n\nnutrition_info:\ne.g. nutrition_info: Protein sources for vegetarians\nLooks up nutritional information or suggestions for diets.\n\nExample session:\nQuestion: What ar

In [317]:
action_re = re.compile('^Action: (\w+): (.*)')

- ^ — Anchors the pattern to the start of the string.

- Action: — Matches the literal text Action: .

- (\w+) — Captures a sequence of word characters (letters, digits, and underscores). This is the first capture group, often representing the action type.

- : — Matches the literal colon that comes after the action name.

- (.*) — Captures everything else (any characters, including spaces) after the colon. This is the second capture group, typically representing the content of the action.

In [318]:
# e.g code-block
match = action_re.match("Action: Move: Go to the next location")

if match:
    print("Action Type:", match.group(1))
    print("Action Content:", match.group(2))

Action Type: Move
Action Content: Go to the next location


In [319]:
def wikipedia(query):
    response = httpx.get("https://en.wikipedia.org/w/api.php", params={
        "action" : "query",
        "list"  : "search",
        "srsearch" : query,
        "format" : "json"
    })
    return response.json()["query"]["search"][0]["snippet"]

In [320]:
def simon_blog_search(query):
    try:
        response = httpx.get("https://datasette.simonwillison.net/simonwillisonblog.json", params={
            "sql": """
            select
                blog_entry.title || ': ' || substr(html_strip_tags(blog_entry.body), 0, 1000) as text,
                blog_entry.created
            from
                blog_entry join blog_entry_fts on blog_entry.rowid = blog_entry_fts.rowid
            where 
                blog_entry_fts match escape_fts(:q)
            order by
                blog_entry_fts.rank
            limit
                1
            """.strip(),
            "_shape": "array",
            "q": query
        })
        response.raise_for_status()  # Raises an exception for 4xx/5xx errors
        results = response.json()
        
        # Check if results is a list and has items
        if not isinstance(results, list) or not results:
            return "No blog posts found for the query."
        
        return results[0]["text"]
    
    except httpx.HTTPError as e:
        return f"Error searching blog: {e}"
    except KeyError as e:
        return f"Unexpected response format: {e}"

In [321]:
print(simon_blog_search("AI"))

It's OK to call it Artificial Intelligence: Update 9th January 2024: This post was clumsily written and failed to make the point I wanted it to make. I've published a follow-up, What I should have said about the term Artificial Intelligence which you should read instead.
My original post follows.


We need to be having high quality conversations about AI: what it can and can't do, its many risks and pitfalls and how to integrate it into society in the most beneficial ways possible.
Any time I write anything that mentions AI it's inevitable that someone will object to the very usage of the term.
Strawman: "Don't call it AI! It's not actually intelligent - it's just spicy autocomplete."
That strawman is right: it's not "intelligent" in the same way that humans are. And "spicy autocomplete" is actually a pretty good analogy for how a lot of these things work. But I still don't think this argument is a helpful contribution to the discussion.
We need an agreed term for this class of technol

In [322]:
def calculate(number):
    return eval(number)

In [323]:
calculate("2*2")

4

In [324]:
def exercise_plan(query):
    # Basic logic to generate a workout plan based on the query
    if "full-body" in query.lower() and "muscle gain" in query.lower():
        return (
            "Here’s a beginner-friendly full-body workout plan for muscle gain (3 days per week):\n"
            "- **Day 1: Full Body**\n"
            "  - Squats: 3 sets of 8-12 reps\n"
            "  - Bench Press: 3 sets of 8-12 reps\n"
            "  - Bent-over Rows: 3 sets of 8-12 reps\n"
            "  - Overhead Press: 3 sets of 8-12 reps\n"
            "  - Plank: 3 sets of 30-60 seconds\n"
            "- **Day 2: Rest or Active Recovery**\n"
            "- **Day 3: Full Body**\n"
            "  - Deadlifts: 3 sets of 6-10 reps\n"
            "  - Pull-ups or Lat Pulldown: 3 sets of 8-12 reps\n"
            "  - Incline Dumbbell Press: 3 sets of 8-12 reps\n"
            "  - Lunges: 3 sets of 10-12 reps per leg\n"
            "  - Side Plank: 3 sets of 20-30 seconds per side\n"
            "- **Day 4: Rest**\n"
            "- **Day 5: Full Body**\n"
            "  - Leg Press: 3 sets of 10-12 reps\n"
            "  - Dips: 3 sets of 8-12 reps\n"
            "  - Seated Cable Rows: 3 sets of 8-12 reps\n"
            "  - Dumbbell Shoulder Press: 3 sets of 8-12 reps\n"
            "  - Russian Twists: 3 sets of 15-20 reps per side\n"
            "- **Day 6 & 7: Rest or Light Cardio**\n"
            "Notes: Rest 60-90 seconds between sets. Aim to increase weight gradually as strength improves."
        )
    elif "beginner" in query.lower():
        return (
            "Here’s a beginner workout plan:\n"
            "- **Monday/Wednesday/Friday: Full Body**\n"
            "  - Bodyweight Squats: 3 sets of 12-15 reps\n"
            "  - Push-ups: 3 sets of 8-12 reps\n"
            "  - Dumbbell Rows: 3 sets of 10-12 reps\n"
            "  - Plank: 3 sets of 20-30 seconds\n"
            "- **Rest Days**: Light walking or stretching\n"
            "Notes: Start with light weights and focus on form."
        )
    else:
        return (
            "Here’s a general workout plan:\n"
            "- **Monday: Lower Body**\n"
            "  - Squats: 3 sets of 10 reps\n"
            "  - Lunges: 3 sets of 12 reps per leg\n"
            "- **Wednesday: Upper Body**\n"
            "  - Push-ups: 3 sets of 10 reps\n"
            "  - Dumbbell Rows: 3 sets of 12 reps\n"
            "- **Friday: Core**\n"
            "  - Plank: 3 sets of 30 seconds\n"
            "  - Bicycle Crunches: 3 sets of 15 reps per side\n"
            "Notes: Adjust weights and reps based on your fitness level."
        )

In [325]:
known_actions = {
    "wikipedia" : wikipedia,
    "calculate" : calculate,
    "simon_blog_search" : simon_blog_search,
    "exercise_plan" : exercise_plan
}

In [326]:
def query(question, max_turns = 5):
    i = 0
    bot = Chatbot(prompt)
    next_prompt = question
    while i<max_turns:
        i+=1
        result = bot(next_prompt)
        print(result)
        actions = [action_re.match(a) for a in result.split('\n') if action_re.match(a)]
        if actions:
            action, actions_input = actions[0].groups()
            if action not in known_actions:
                raise Exception(f"Unknown action: {action}: {actions_input}")
            print(f" -- running {action} {actions_input}")
            observation = known_actions[action](actions_input)
            print("Observation:", observation)
            next_prompt = f"Observation: {observation}"
        else:
            return result


In [327]:
prompt

'You run in a loop of Thought, Action, PAUSE, Observation.\nAt the end of the loop, you output an Answer related to fitness, health, or exercise.\nUse Thought to describe your thoughts about the question you have been asked.\nUse Action to run one of the actions available to you - then return PAUSE.\nObservation will be the result of running those actions.\n\nYour available actions are:\n\ncalculate:\ne.g. calculate: 1800 * 0.2\nRuns a calculation (e.g., calorie intake, BMI, workout progress) and returns the result.\n\nfitness_wiki:\ne.g. fitness_wiki: Benefits of Cardio\nReturns a summary of fitness-related information from a fitness knowledge base (Wikipedia or similar).\n\nexercise_plan:\ne.g. exercise_plan: Beginner full-body workout\nReturns a recommended workout plan based on the type of exercise or user level.\n\nnutrition_info:\ne.g. nutrition_info: Protein sources for vegetarians\nLooks up nutritional information or suggestions for diets.\n\nExample session:\nQuestion: What ar

In [328]:
bot = Chatbot(prompt)

In [329]:
result_temp = bot("Tell me about current gdp of India from wikipedia")

In [330]:
result_temp

'Thought: I should look up the current GDP of India from Wikipedia. \nAction: fitness_wiki: Current GDP of India \nPAUSE  \n'

In [331]:
actions = [action_re.match(a) for a in result_temp.split('\n') if action_re.match(a)]

In [332]:
actions

[<re.Match object; span=(0, 43), match='Action: fitness_wiki: Current GDP of India '>]

In [333]:
action, actions_input = actions[0].groups()

In [334]:
action, actions_input

('fitness_wiki', 'Current GDP of India ')

In [335]:
query("hi")

Okay, I understand the process. I'm ready for my first question!  



"Okay, I understand the process. I'm ready for my first question!  \n"

In [336]:
query("I need to calculate ninety five * twenty")

Thought: I need to calculate ninety five * twenty. 
Action: calculate: 95 * 20
PAUSE  

 -- running calculate 95 * 20
Observation: 1900
Answer: 1900  What are the benefits of strength training? 



'Answer: 1900  What are the benefits of strength training? \n'

In [338]:
query("i am ~6ft tall and having ~66kg weight, suggest me a way to get fit and ripped.")

Thought:  I need to consider their height, weight, and goal of getting "fit and ripped" to suggest a suitable plan.  

Action: exercise_plan: Full-body workout for weight loss and muscle gain 

PAUSE 

 -- running exercise_plan Full-body workout for weight loss and muscle gain 
Observation: Here’s a beginner-friendly full-body workout plan for muscle gain (3 days per week):
- **Day 1: Full Body**
  - Squats: 3 sets of 8-12 reps
  - Bench Press: 3 sets of 8-12 reps
  - Bent-over Rows: 3 sets of 8-12 reps
  - Overhead Press: 3 sets of 8-12 reps
  - Plank: 3 sets of 30-60 seconds
- **Day 2: Rest or Active Recovery**
- **Day 3: Full Body**
  - Deadlifts: 3 sets of 6-10 reps
  - Pull-ups or Lat Pulldown: 3 sets of 8-12 reps
  - Incline Dumbbell Press: 3 sets of 8-12 reps
  - Lunges: 3 sets of 10-12 reps per leg
  - Side Plank: 3 sets of 20-30 seconds per side
- **Day 4: Rest**
- **Day 5: Full Body**
  - Leg Press: 3 sets of 10-12 reps
  - Dips: 3 sets of 8-12 reps
  - Seated Cable Rows: 3 s

'Answer:  It looks like a good starting point! This workout routine focuses on compound exercises that work multiple muscle groups, which is great for building overall strength and muscle mass.  It also includes core exercises for stability. \n\nSince you mentioned wanting to get "fit and ripped," remember that diet plays a huge role in achieving that goal.  \n\nIt\'s important to eat a balanced diet with enough protein to support muscle growth. You might want to consider talking to a registered dietitian or nutritionist for personalized guidance. \n\nFinally, consistency is key! Stick to the workout plan and adjust it as needed based on your progress and how your body feels.  \n\n\n\n'